# Filip Kucia, Bartosz Grabek

# 1. identifying product data (categories and brands) with LLM

## google/pegasus-xsum, really random outpus to be honest

In [6]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("google/pegasus-xsum")
model = AutoModelForSeq2SeqLM.from_pretrained("google/pegasus-xsum")

# Function to extract category and brand
def extract_attributes(text):
    prompt = f"""Extract the product category and brand from the following product description:

{text}

Format your response as:
Category: <category>
Brand: <brand>"""
    # Tokenize the input prompt
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding='longest')
    # Generate the model's output
    outputs = model.generate(**inputs, max_length=100)
    # Decode and return the response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Sample product data
product_data = {
    "text": [
        "Neutrogena SPF50 Sunscreen - Pore-Friendly Protection",
        "Dreft Baby Powder Detergent - Gentle on Baby Clothes",
        "Arm & Hammer Powder Detergent - Great Value, Lasts Forever",
        "Neutrogena Ultra Sheer Dry-Touch Sunscreen SPF70 - Non-greasy",
        "Tide Liquid Detergent - Deep Clean",
    ]
}

# Extract and print attributes for each product description
for text in product_data["text"]:
    response = extract_attributes(text)
    print(f"Product Description: {text}")
    print(response)
    print("-----")


tokenizer_config.json:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.52M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/259 [00:00<?, ?B/s]

Product Description: Neutrogena SPF50 Sunscreen - Pore-Friendly Protection
What is your favourite sunscreen?
-----
Product Description: Dreft Baby Powder Detergent - Gentle on Baby Clothes
Dreft Baby Powder Detergent - Gentle on Baby Clothes
-----
Product Description: Arm & Hammer Powder Detergent - Great Value, Lasts Forever
Arm & Hammer Powder Detergent - Great Value, Lasts Forever
-----
Product Description: Neutrogena Ultra Sheer Dry-Touch Sunscreen SPF70 - Non-greasy
Neutrogena Ultra Sheer Dry-Touch Sunscreen SPF70 - Non-greasy
-----
Product Description: Tide Liquid Detergent - Deep Clean
What is your favourite product from the Tide range?
-----


# gpt2, ignored template and just commented on the text desciption, describing them more

In [19]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load the tokenizer and model for GPT-2
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")

# Function to extract category and brand using a prompt
def extract_attributes(text):
    """
    This function takes a product description as input and returns the
    extracted product category and brand using GPT-2.

    Args:
        text (str): The product description.

    Returns:
        response (str): The model's response containing the category and brand.
    """
    # Provide a clear instruction to the model
    prompt = f"""Extract the product category and brand from the following product description:

Product Description: {text}

Format your response as:
Category: <category>
Brand: <brand>"""

    # Tokenize the input prompt
    inputs = tokenizer.encode(prompt, return_tensors="pt")

    # Generate the model's output
    outputs = model.generate(
        inputs,
        max_length=100,  # Increase max length for sufficient output
        num_beams=5,  # Use beam search for better text generation
        early_stopping=True,  # Stop early if the model completes the task
        no_repeat_ngram_size=2  # Avoid repetition
    )

    # Decode the model's output into human-readable text
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Only keep the relevant part of the response (after "Category:")
    if "Category:" in response:
        response = response.split("Category:")[1]  # Extract the part after "Category:"
        response = "Category:" + response.strip()  # Add "Category:" back and strip whitespace

    return response

# Sample product data
product_data = {
    "text": [
        "Neutrogena SPF50 Sunscreen - Pore-Friendly Protection",
        "Dreft Baby Powder Detergent - Gentle on Baby Clothes",
        "Arm & Hammer Powder Detergent - Great Value, Lasts Forever",
        "Neutrogena Ultra Sheer Dry-Touch Sunscreen SPF70 - Non-greasy",
        "Tide Liquid Detergent - Deep Clean",
    ]
}

# Extract and print attributes for each product description
for text in product_data["text"]:
    response = extract_attributes(text)
    print(f"Product Description: {text}")
    print(response)
    print("-----")


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Product Description: Dreft Baby Powder Detergent - Gentle on Baby Clothes
Category:<category>
Brand: <brand>


Dreft baby powder detergent is a gentle, non-comedogenic, anti-bacterial, and hydrating product. It is formulated to be used in a wide range of baby clothes, including baby diapers, baby wipes and baby
-----


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Product Description: Arm & Hammer Powder Detergent - Great Value, Lasts Forever
Category:<category>
Brand: <brand>


This product is not available for purchase in the United States or Canada. Please contact us for more information.
-----


KeyboardInterrupt: 

# distilgpt2 with prompt, had i would say hallucination going into repeating itself with brand brand

In [16]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load the tokenizer and model for DistilGPT2 (causal language model)
tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
model = AutoModelForCausalLM.from_pretrained("distilgpt2")

# Function to extract category and brand using a prompt
def extract_attributes(text):
    """
    This function takes a product description as input and returns the
    extracted product category and brand using a causal language model (DistilGPT2).

    Args:
        text (str): The product description.

    Returns:
        response (str): The model's response containing the category and brand.
    """
    prompt = f"""Extract the product category and brand from the following product description:

{text}

Format your response as:
Category: <category>
Brand: <brand>"""

    # Tokenize the input prompt
    inputs = tokenizer(prompt, return_tensors="pt")

    # Generate the model's output
    outputs = model.generate(**inputs, max_length=500, pad_token_id=tokenizer.eos_token_id)

    # Decode the model's output into human-readable text
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return response

# Sample product data
product_data = {
    "text": [
        "Neutrogena SPF50 Sunscreen - Pore-Friendly Protection",
        "Dreft Baby Powder Detergent - Gentle on Baby Clothes",
        "Arm & Hammer Powder Detergent - Great Value, Lasts Forever",
        "Neutrogena Ultra Sheer Dry-Touch Sunscreen SPF70 - Non-greasy",
        "Tide Liquid Detergent - Deep Clean",
    ]
}

# Extract and print attributes for each product description
for text in product_data["text"]:
    response = extract_attributes(text)
    print(f"Product Description: {text}")
    print(response)
    print("-----")


Product Description: Neutrogena SPF50 Sunscreen - Pore-Friendly Protection
Extract the product category and brand from the following product description:

Neutrogena SPF50 Sunscreen - Pore-Friendly Protection

Format your response as:
Category: <category>
Brand: <brand>
Brand: <brand>
Brand: <brand>
Brand: <brand>
Brand: <brand>
Brand: <brand>
Brand: <brand>
Brand: <brand>
Brand: <brand>
Brand: <brand>
Brand: <brand>
Brand: <brand>
Brand: <brand>
Brand: <brand>
Brand: <brand>
Brand: <brand>
Brand: <brand>
Brand: <brand>
Brand: <brand>
Brand: <brand>
Brand: <brand>
Brand: <brand>
Brand: <brand>
Brand: <brand>
Brand: <brand>
Brand: <brand>
Brand: <brand>
Brand: <brand>
Brand: <brand>
Brand: <brand>
Brand: <brand>
Brand: <brand>
Brand: <brand>
Brand: <brand>
Brand: <brand>
Brand: <brand>
Brand: <brand>
Brand: <brand>
Brand: <brand>
Brand: <brand>
Brand: <brand>
Brand: <brand>
Brand: <brand>
Brand: <brand>
Brand: <brand>
Brand: <brand>
Brand: <brand>
Brand: <brand>
Brand: <brand>
Brand: <b

KeyboardInterrupt: 

## RoBERTa MLM, prompt

In [13]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch

# Load the distilroberta-base model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")
model = AutoModelForMaskedLM.from_pretrained("distilroberta-base")

# Function to extract category and brand using masked token prediction
def extract_attributes(text):
    """
    This function takes a product description as input and returns the
    extracted product category and brand using the distilroberta-base model.

    Args:
        text (str): The product description.

    Returns:
        response (str): Extracted category and brand.
    """
    # Insert <mask> tokens in place of where we expect the brand and category
    prompt = f"""Extract the product category and brand from the following product description:

{text}

Category: <mask>
Brand: <mask>"""

    # Tokenize the input prompt
    inputs = tokenizer(prompt, return_tensors="pt")

    # Get the model's predictions
    with torch.no_grad():
        outputs = model(**inputs)

    # Find the top predictions for the masked tokens
    mask_token_index = torch.where(inputs.input_ids == tokenizer.mask_token_id)[1]
    logits = outputs.logits

    # Decode the predicted tokens for the category and brand
    predicted_token_ids = logits[0, mask_token_index].argmax(dim=-1)
    predicted_tokens = tokenizer.decode(predicted_token_ids)

    # Split the predicted tokens into category and brand based on the mask positions
    category, brand = predicted_tokens.split()

    response = f"Category: {category}\nBrand: {brand}"

    return response

# Sample product data
product_data = {
    "text": [
        "Neutrogena SPF50 Sunscreen - Pore-Friendly Protection",
        "Dreft Baby Powder Detergent - Gentle on Baby Clothes",
        "Arm & Hammer Powder Detergent - Great Value, Lasts Forever",
        "Neutrogena Ultra Sheer Dry-Touch Sunscreen SPF70 - Non-greasy",
        "Tide Liquid Detergent - Deep Clean",
    ]
}

# Extract and print attributes for each product description
for text in product_data["text"]:
    response = extract_attributes(text)
    print(f"Product Description: {text}")
    print(response)
    print("-----")


Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Product Description: Neutrogena SPF50 Sunscreen - Pore-Friendly Protection
Category: Accessories
Brand: Beauty
-----
Product Description: Dreft Baby Powder Detergent - Gentle on Baby Clothes
Category: Beauty
Brand: Baby
-----
Product Description: Arm & Hammer Powder Detergent - Great Value, Lasts Forever
Category: Products
Brand: Accessories
-----
Product Description: Neutrogena Ultra Sheer Dry-Touch Sunscreen SPF70 - Non-greasy
Category: Accessories
Brand: Accessories
-----
Product Description: Tide Liquid Detergent - Deep Clean
Category: Products
Brand: Products
-----


# Smaller model from google Flan-t5

In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

# Function to extract category and brand
def extract_attributes(text):
    prompt = f"""Extract the product category and brand from the following product description:

{text}

Format your response as:
Category: <category>
Brand: <brand>"""
    # Tokenize the input prompt
    inputs = tokenizer(prompt, return_tensors="pt")
    # Generate the model's output
    outputs = model.generate(**inputs, max_length=50)
    # Decode and return the response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Sample product data
product_data = {
    "text": [
        "Neutrogena SPF50 Sunscreen - Pore-Friendly Protection",
        "Dreft Baby Powder Detergent - Gentle on Baby Clothes",
        "Arm & Hammer Powder Detergent - Great Value, Lasts Forever",
        "Neutrogena Ultra Sheer Dry-Touch Sunscreen SPF70 - Non-greasy",
        "Tide Liquid Detergent - Deep Clean",
    ]
}

# Extract and print attributes for each product description
for text in product_data["text"]:
    response = extract_attributes(text)
    print(f"Product Description: {text}")
    print(response)
    print("-----")

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Product Description: Neutrogena SPF50 Sunscreen - Pore-Friendly Protection
Neutrogena
-----
Product Description: Dreft Baby Powder Detergent - Gentle on Baby Clothes
Dreft Baby Powder Detergent - Gentle on Baby Clothes
-----
Product Description: Arm & Hammer Powder Detergent - Great Value, Lasts Forever
Arm & Hammer Powder Detergent - Great Value, Lasts Forever
-----
Product Description: Neutrogena Ultra Sheer Dry-Touch Sunscreen SPF70 - Non-greasy
Neutrogena
-----
Product Description: Tide Liquid Detergent - Deep Clean
Tide Liquid Detergent - Deep Clean
-----


## The best results from prompt template are from Flan-t5

## Few shot learning with setfit library

In [2]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [7]:
from setfit import SetFitModel, Trainer, TrainingArguments, sample_dataset
from datasets import Dataset

# Step 1: Define a few-shot product dataset (Example format)
# You can expand this with more categories, brands, and attributes
product_data = {
    "text": [
        "Neutrogena SPF50 Sunscreen - Pore-Friendly Protection",
        "Dreft Baby Powder Detergent - Gentle on Baby Clothes",
        "Arm & Hammer Powder Detergent - Great Value, Lasts Forever",
        "Neutrogena Ultra Sheer Dry-Touch Sunscreen SPF70 - Non-greasy",
        "Tide Liquid Detergent - Deep Clean",
    ],
    "category": [
        "Sunscreen",  # Product category
        "Laundry Detergent",  # Product category
        "Laundry Detergent",  # Product category
        "Sunscreen",  # Product category
        "Laundry Detergent"  # Product category
    ],
    "brand": [
        "Neutrogena",  # Product brand
        "Dreft",  # Product brand
        "Arm & Hammer",  # Product brand
        "Neutrogena",  # Product brand
        "Tide",  # Product brand
    ]
}

# Step 2: Prepare the dataset as a Huggingface Dataset
# We combine both category and brand for multi-label classification, but can split them if needed.
labels = ["category", "brand"]  # Use category and brand as labels

# Create a "label" column combining both the category and brand for simplicity
product_data["label"] = [f"{cat}, {brand}" for cat, brand in zip(product_data["category"], product_data["brand"])]
del product_data["category"]  # Remove the category column (now integrated into label)
del product_data["brand"]  # Remove the brand column (now integrated into label)

# Create a dataset
dataset = Dataset.from_dict(product_data)

# Step 3: Initialize a SetFit model and define labels (category, brand combinations)
model = SetFitModel.from_pretrained("BAAI/bge-small-en-v1.5")

# We can dynamically add all the possible category/brand combinations as labels
# For example: ["Sunscreen, Neutrogena", "Laundry Detergent, Dreft", ...]
unique_labels = list(set(product_data["label"]))
model.labels = unique_labels

# Step 4: Sample few-shot data for training
train_dataset = sample_dataset(dataset, label_column="label", num_samples=2)  # 2 examples per class for few-shot
test_dataset = dataset  # You can split this into separate test dataset if needed

# Step 5: Prepare training arguments
args = TrainingArguments(
    batch_size=16,  # Adjust based on your data
    num_epochs=15,  # Use fewer epochs for small data, more for large data
)

# Step 6: Initialize the Trainer with the model and training data
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
)

# Step 7: Train the model on the few-shot data
trainer.train()

# Step 8: Evaluate the model (optional, assuming a test set)
metrics = trainer.evaluate(test_dataset)
print(metrics)

# Step 9: Perform inference on new product descriptions or reviews
new_products = [
    "Neutrogena SPF50 sunscreen is perfect for daily use and doesn’t clog pores.",
    "Dreft is my favorite detergent for baby clothes; it's so gentle!",
    "Tide gets out the toughest stains!"
]

# Predict the category and brand
predictions = model.predict(new_products)

for product, prediction in zip(new_products, predictions):
    print(f"Product: {product}\nPredicted Label (Category, Brand): {prediction}\n")


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is depreca

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

***** Running training *****
  Num unique pairs = 18
  Batch size = 16
  Num epochs = 15


Step,Training Loss
1,0.178900


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

***** Running evaluation *****


{'accuracy': 1.0}
Product: Neutrogena SPF50 sunscreen is perfect for daily use and doesn’t clog pores.
Predicted Label (Category, Brand): Sunscreen, Neutrogena

Product: Dreft is my favorite detergent for baby clothes; it's so gentle!
Predicted Label (Category, Brand): Laundry Detergent, Dreft

Product: Tide gets out the toughest stains!
Predicted Label (Category, Brand): Sunscreen, Neutrogena



# Perfect results but only on 3 never seen examples

In [8]:
import os
from setfit import SetFitModel, Trainer, TrainingArguments, sample_dataset
from datasets import Dataset

# Disable Weights and Biases (wandb)
os.environ["WANDB_DISABLED"] = "true"

# Step 1: Define a few-shot product dataset (Example format)
product_data = {
    "text": [
        "Neutrogena SPF50 Sunscreen - Pore-Friendly Protection",
        "Dreft Baby Powder Detergent - Gentle on Baby Clothes",
        "Arm & Hammer Powder Detergent - Great Value, Lasts Forever",
        "Neutrogena Ultra Sheer Dry-Touch Sunscreen SPF70 - Non-greasy",
        "Tide Liquid Detergent - Deep Clean",
    ],
    "category": [
        "Sunscreen",  # Product category
        "Laundry Detergent",  # Product category
        "Laundry Detergent",  # Product category
        "Sunscreen",  # Product category
        "Laundry Detergent"  # Product category
    ],
    "brand": [
        "Neutrogena",  # Product brand
        "Dreft",  # Product brand
        "Arm & Hammer",  # Product brand
        "Neutrogena",  # Product brand
        "Tide",  # Product brand
    ]
}

# Combine both category and brand into a single label
product_data["label"] = [f"{cat}, {brand}" for cat, brand in zip(product_data["category"], product_data["brand"])]
del product_data["category"]
del product_data["brand"]

# Step 2: Prepare the dataset as a Huggingface Dataset
dataset = Dataset.from_dict(product_data)

# Step 3: Initialize a new SetFit model with `paraphrase-mpnet-base-v2`
model = SetFitModel.from_pretrained("sentence-transformers/paraphrase-mpnet-base-v2")

# Set labels for the model (category, brand combinations)
unique_labels = list(set(product_data["label"]))
model.labels = unique_labels

# Step 4: Sample few-shot data for training
train_dataset = sample_dataset(dataset, label_column="label", num_samples=2)
test_dataset = dataset  # Using full dataset as test for simplicity

# Step 5: Prepare training arguments (with wandb disabled)
args = TrainingArguments(
    batch_size=8,
    num_epochs=5,
    report_to=["none"]  # Disabling logging to wandb
)

# Step 6: Initialize the Trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
)

# Step 7: Train the model
trainer.train()

# Step 8: Evaluate the model
metrics = trainer.evaluate(test_dataset)
print(metrics)

# Step 9: Predict on new product descriptions
new_products = [
    "Neutrogena SPF50 sunscreen is perfect for daily use and doesn’t clog pores.",
    "Dreft is my favorite detergent for baby clothes; it's so gentle!",
    "Tide gets out the toughest stains!"
]

predictions = model.predict(new_products)

for product, prediction in zip(new_products, predictions):
    print(f"Product: {product}\nPredicted Label (Category, Brand): {prediction}\n")


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

***** Running training *****
  Num unique pairs = 18
  Batch size = 8
  Num epochs = 5


Step,Training Loss
1,0.072400


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

***** Running evaluation *****


{'accuracy': 1.0}
Product: Neutrogena SPF50 sunscreen is perfect for daily use and doesn’t clog pores.
Predicted Label (Category, Brand): Sunscreen, Neutrogena

Product: Dreft is my favorite detergent for baby clothes; it's so gentle!
Predicted Label (Category, Brand): Laundry Detergent, Dreft

Product: Tide gets out the toughest stains!
Predicted Label (Category, Brand): Laundry Detergent, Tide



# Trying with bigger dataset

In [26]:
product_data = {
    "text": [
        # Thermal Mugs
        "Contigo Workday Travel Mug – Keeps Coffee Hot for Hours!",
        "Zojirushi Sleek Travel Mug – Perfect Fit for Car Holders",
        "Hydro Flask Lightweight Insulation Mug – Stay Warm for Hours",
        "Yeti Thermal Mug – No More Cold Coffee!",
        "Contigo All-Day Heat Retention Mug – Ideal for Travel",
        "Contigo Leak-Proof Mug – Toss in Your Bag with Confidence",
        "Zojirushi Scalding Hot Coffee Mug – Best Insulation Yet",
        "Thermos Ultimate Heat Mug – Keeps Drinks Piping Hot",
        "Stanley Classic Vacuum Mug – Durable for Outdoor Adventures",
        "BrüMate Shatterproof Mug – Ideal for Iced Coffee Too",
        "Contigo SmartLock Mug – Never Spill Again!",
        "Ember Temperature Control Mug – Adjust Heat with Your Phone",
        "Hydro Flask All-Day Heat Mug – Built for Comfort",
        "Thermos High-Performance Mug – Best for Long Commutes",
        "CamelBak Insulated Mug – Perfect for Hikes",

        # Dishwasher Detergents
        "Cascade Sparkling Clean Detergent – Your Dishes Will Shine",
        "Finish Detergent – Tough on Grease, No Residue Left",
        "Cascade Platinum Detergent – Pricey, But Worth It for Results",
        "Finish Quantum Detergent – Stubborn Food Stains Gone",
        "Cascade Complete Detergent – No Pre-Rinsing Needed for Grease",
        "Palmolive Eco+ Detergent – Gentle on Hands, Tough on Dishes",
        "Seventh Generation Dishwasher Detergent – Safe and Effective",
        "Method Smarty Dish Detergent – Great for Families",
        "Finish Powerball Detergent – Excellent for Hard Water Areas",
        "Cascade Pure Essentials Detergent – No Harsh Chemicals",
        "Kirkland Signature Dishwasher Detergent – Budget-Friendly, Great Results",
        "Ecover Zero Detergent – Ideal for Sensitive Skin",
        "Cascade ActionPacs Detergent – Easy to Use, Powerful Cleaning",
        "Finish Jet-Dry Detergent – Sparkling Finish Every Time",
        "Eco-Me Natural Dishwasher Detergent – Safe and Non-Toxic",

        # Sunscreens
        "Neutrogena Daily Sunscreen – Quick Absorption, No Grease",
        "Coppertone Suncream – Strong Scent, Strong Protection",
        "La Roche-Posay Sensitive Skin Sunscreen – No Breakouts",
        "CeraVe Lightweight Sunscreen – Perfect Under Makeup",
        "Banana Boat Beach-Saver Sunscreen – No Burns, Just Fun",
        "Hawaiian Tropic Waterproof Sunscreen – Pool Day Essential",
        "Supergoop Premium Sunscreen – Worth Every Penny",
        "Neutrogena Face & Body Sunscreen – All-Purpose Protection",
        "EltaMD Pore-Friendly Sunscreen – Protection Without Clogging",
        "La Roche-Posay Suncream – The Best in Sun Protection",
        "Neutrogena Sunscreen – Soft, Protected Skin All Day",
        "Aveeno Protect + Hydrate Sunscreen – Moisturizing and Non-Greasy",
        "Blue Lizard Mineral Sunscreen – Reef-Safe and Effective",
        "Banana Boat Kids Sunscreen – No Tears Formula",
        "Coppertone Sport Sunscreen – Long-Lasting for Active Days",
        "Sun Bum Moisturizing Sunscreen – Smells Great, Feels Better",
        "ThinkSport Safe Sunscreen – Great for Outdoor Sports",
        "Badger Balm Zinc Sunscreen – Organic and Water-Resistant",
        "Alba Botanica Sensitive Sunscreen – Ideal for Delicate Skin",
        "Bullfrog Mosquito Repellent Sunscreen – Two-in-One Protection",

        # Powder Detergents for Laundry
        "Tide Powder Detergent – Fresh, Clean Clothes Every Time",
        "Ariel Powder Detergent – Whites Brighter, Even in Cold Water",
        "Persil Powder Detergent – Powerful Stain Removal",
        "OMO Powder Detergent – Leaves Residue on Dark Clothes",
        "Seventh Generation Powder Detergent – Great for Sensitive Skin",
        "Ecover Eco-Friendly Detergent – Perfect for the Eco-Conscious",
        "Gain Powder Detergent – Fresh-Smelling Laundry for Days",
        "Arm & Hammer Powder Detergent – Great Value, Lasts Forever",
        "Dreft Baby Powder Detergent – Gentle on Baby Clothes",
        "Surf Tropical Breeze Detergent – Leaves Clothes Smelling Amazing",
        "All Mighty Powder Detergent – Free of Dyes, Gentle on Skin",
        "Purex Powder Detergent – Affordable, Effective on Stains",
        "Charlie’s Soap Laundry Powder – Safe for All Fabrics",
        "Foca Laundry Detergent – Biodegradable and Phosphate-Free",
        "BioKleen Premium Powder Detergent – Non-Toxic and Plant-Based",
        "Sun Powder Detergent – Brightens Colors, Fights Tough Stains",
        "Bold 2in1 Powder Detergent – Softens and Cleans in One Go",
        "Oxydol Laundry Detergent – Contains Oxygen Bleach for Extra Cleaning",
        "Kirkland Signature Ultra Powder Detergent – Bulk Buy, Great Results",
        "Fab Powder Detergent – Clean Laundry with a Light Fragrance"
    ],
    "category": [
        # Thermal Mugs
        "Thermal Mug", "Thermal Mug", "Thermal Mug", "Thermal Mug", "Thermal Mug",
        "Thermal Mug", "Thermal Mug", "Thermal Mug", "Thermal Mug", "Thermal Mug",
        "Thermal Mug", "Thermal Mug", "Thermal Mug", "Thermal Mug", "Thermal Mug",

        # Dishwasher Detergents
        "Dishwasher Detergent", "Dishwasher Detergent", "Dishwasher Detergent", "Dishwasher Detergent", "Dishwasher Detergent",
        "Dishwasher Detergent", "Dishwasher Detergent", "Dishwasher Detergent", "Dishwasher Detergent", "Dishwasher Detergent",
        "Dishwasher Detergent", "Dishwasher Detergent", "Dishwasher Detergent", "Dishwasher Detergent", "Dishwasher Detergent",

        # Sunscreens
        "Sunscreen", "Sunscreen", "Sunscreen", "Sunscreen", "Sunscreen",
        "Sunscreen", "Sunscreen", "Sunscreen", "Sunscreen", "Sunscreen",
        "Sunscreen", "Sunscreen", "Sunscreen", "Sunscreen", "Sunscreen",
        "Sunscreen", "Sunscreen", "Sunscreen", "Sunscreen", "Sunscreen",

        # Powder Detergents for Laundry
        "Powder Detergent", "Powder Detergent", "Powder Detergent", "Powder Detergent", "Powder Detergent",
        "Powder Detergent", "Powder Detergent", "Powder Detergent", "Powder Detergent", "Powder Detergent",
        "Powder Detergent", "Powder Detergent", "Powder Detergent", "Powder Detergent", "Powder Detergent",
        "Powder Detergent", "Powder Detergent", "Powder Detergent", "Powder Detergent", "Powder Detergent"
    ],
    "brand": [
        # Thermal Mugs
        "Contigo", "Zojirushi", "Hydro Flask", "Yeti", "Contigo",
        "Contigo", "Zojirushi", "Thermos", "Stanley", "BrüMate",
        "Contigo", "Ember", "Hydro Flask", "Thermos", "CamelBak",

        # Dishwasher Detergents
        "Cascade", "Finish", "Cascade", "Finish", "Cascade",
        "Palmolive", "Seventh Generation", "Method", "Finish", "Cascade",
        "Kirkland Signature", "Ecover", "Cascade", "Finish", "Eco-Me",

        # Sunscreens
        "Neutrogena", "Coppertone", "La Roche-Posay", "CeraVe", "Banana Boat",
        "Hawaiian Tropic", "Supergoop", "Neutrogena", "EltaMD", "La Roche-Posay",
        "Neutrogena", "Aveeno", "Blue Lizard", "Banana Boat", "Coppertone",
        "Sun Bum", "ThinkSport", "Badger Balm", "Alba Botanica", "Bullfrog",

        # Powder Detergents for Laundry
        "Tide", "Ariel", "Persil", "OMO", "Seventh Generation",
        "Ecover", "Gain", "Arm & Hammer", "Dreft", "Surf",
        "All", "Purex", "Charlie’s Soap", "Foca", "BioKleen",
        "Sun", "Bold", "Oxydol", "Kirkland Signature", "Fab"
    ]
}


In [27]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import os
from setfit import SetFitModel, Trainer, TrainingArguments, sample_dataset
from datasets import Dataset
import pandas as pd


# Step 2: Prepare the dataset as a Huggingface Dataset
df = pd.DataFrame(product_data)
df = df.rename(columns={"category": "label"})
dataset = Dataset.from_pandas(df)

# Step 3: Initialize a new SetFit model with 'paraphrase-mpnet-base-v2'
model = SetFitModel.from_pretrained("sentence-transformers/paraphrase-mpnet-base-v2")

# Set labels for the model (categories)
unique_labels = sorted(list(set(product_data["category"])))
model.model_head.classes_ = unique_labels

# Step 4: Split the dataset into train and test sets (80% train, 20% test)
train_test_split = dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

# Step 5: Prepare training arguments (with wandb disabled)
args = TrainingArguments(
    batch_size=32,
    num_epochs=3,
    report_to=["none"]  # Disabling logging to wandb
)

# Step 6: Initialize the Trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
)

# Step 7: Train the model
trainer.train()

# Step 8: Evaluate the model
test_texts = test_dataset["text"]  # Product descriptions for testing
test_labels = test_dataset["label"]  # Actual categories for testing

# Map labels to indices for evaluation
label_to_index = {label: idx for idx, label in enumerate(unique_labels)}
test_label_indices = [label_to_index[label] for label in test_labels]

# Get predictions for the test set
preds = model.predict(test_texts)

# Ensure predictions are also in terms of numerical indices if they are strings
preds = [label_to_index[pred] if isinstance(pred, str) else pred for pred in preds]

# Generate classification report
report = classification_report(test_label_indices, preds, target_names=unique_labels)

# Display the classification report
print("Classification Report:\n", report)

# Step 9: Print predictions for the test set
print("\nTest Set Predictions:")
for text, actual_label, prediction_idx in zip(test_texts, test_labels, preds):
    predicted_label = unique_labels[prediction_idx]
    print(f"Product: {text}\nActual Category: {actual_label}\nPredicted Category: {predicted_label}\n")

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Map:   0%|          | 0/56 [00:00<?, ? examples/s]

***** Running training *****
  Num unique pairs = 2332
  Batch size = 32
  Num epochs = 3


Step,Training Loss
1,0.184000
50,0.071500
100,0.006000
150,0.001100
200,0.000800


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Classification Report:
                       precision    recall  f1-score   support

Dishwasher Detergent       1.00      0.50      0.67         4
    Powder Detergent       0.71      1.00      0.83         5
           Sunscreen       1.00      1.00      1.00         3
         Thermal Mug       1.00      1.00      1.00         2

            accuracy                           0.86        14
           macro avg       0.93      0.88      0.88        14
        weighted avg       0.90      0.86      0.85        14


Test Set Predictions:
Product: Foca Laundry Detergent – Biodegradable and Phosphate-Free
Actual Category: Powder Detergent
Predicted Category: Powder Detergent

Product: Cascade Pure Essentials Detergent – No Harsh Chemicals
Actual Category: Dishwasher Detergent
Predicted Category: Powder Detergent

Product: La Roche-Posay Sensitive Skin Sunscreen – No Breakouts
Actual Category: Sunscreen
Predicted Category: Sunscreen

Product: Seventh Generation Powder Detergent – Great 

In [28]:
# Step 10: Predict on new product descriptions
new_products = [
    "Neutrogena SPF50 sunscreen is perfect for daily use and doesn’t clog pores.",
    "Dreft is my favorite detergent for baby clothes; it's so gentle!",
    "Tide gets out the toughest stains!"
]

# Predictions for new products
new_preds = model.predict(new_products)

# If predictions are in string form, map them to their corresponding numerical indices
new_preds_indices = [label_to_index[pred] if isinstance(pred, str) else pred for pred in new_preds]

# Mapping predictions to human-readable category format
print("\nNew Products Predictions:")
for product, prediction_idx in zip(new_products, new_preds_indices):
    predicted_category = unique_labels[prediction_idx]
    print(f"Product: {product}\nPredicted Category: {predicted_category}\n")



New Products Predictions:
Product: Neutrogena SPF50 sunscreen is perfect for daily use and doesn’t clog pores.
Predicted Category: Sunscreen

Product: Dreft is my favorite detergent for baby clothes; it's so gentle!
Predicted Category: Powder Detergent

Product: Tide gets out the toughest stains!
Predicted Category: Powder Detergent



/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# results of few shot prompting are very nice, but we must keep in mind that we had to label them and it will produce only the labels that are in scope

In [6]:
!pip install setfit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.2/75.2 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.1 MB/s eta 0:00:00


#2. define similarity between products from the dataset and reviews
## There are various ways to define similarity between products

# Semantic Textual Similarity (STS)

In [21]:
from sentence_transformers import SentenceTransformer, util

# Load pre-trained model fine-tuned on STS tasks
model = SentenceTransformer('stsb-roberta-large')

# Define product descriptions and reviews
product_descriptions = [
    "Neutrogena SPF50 Sunscreen - Pore-Friendly Protection",
    "Dreft Baby Powder Detergent - Gentle on Baby Clothes",
    "Tide Liquid Detergent - Deep Clean"
]
reviews = [
    "This sunscreen is great for sensitive skin and daily use. It doesn't clog pores and offers great protection!",
    "I love using Dreft for my baby's clothes. It's so gentle and keeps everything smelling fresh."
]

# Compute embeddings using the model
product_embeddings = model.encode(product_descriptions, convert_to_tensor=True)
review_embeddings = model.encode(reviews, convert_to_tensor=True)

# Compute semantic similarity scores using cosine similarity
cosine_sim = util.pytorch_cos_sim(review_embeddings, product_embeddings)

# Output similarity scores
for i, review in enumerate(reviews):
    print(f"Review: {review}")
    for j, product in enumerate(product_descriptions):
        print(f" - {product}: {cosine_sim[i][j]:.4f}")


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.96k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Review: This sunscreen is great for sensitive skin and daily use. It doesn't clog pores and offers great protection!
 - Neutrogena SPF50 Sunscreen - Pore-Friendly Protection: 0.5585
 - Dreft Baby Powder Detergent - Gentle on Baby Clothes: 0.3978
 - Tide Liquid Detergent - Deep Clean: 0.4376
Review: I love using Dreft for my baby's clothes. It's so gentle and keeps everything smelling fresh.
 - Neutrogena SPF50 Sunscreen - Pore-Friendly Protection: 0.3392
 - Dreft Baby Powder Detergent - Gentle on Baby Clothes: 0.6582
 - Tide Liquid Detergent - Deep Clean: 0.4709


# Universal Sentence Encoder (USE):
The Universal Sentence Encoder (USE) is an NLP model developed by Google that produces embeddings that are optimized for measuring semantic similarity between sentences. It can be applied to tasks like question answering, content retrieval, and more.

In [24]:
import tensorflow_hub as hub
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load Universal Sentence Encoder model from TensorFlow Hub
model = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

# Define product descriptions and reviews
product_descriptions = [
    "Neutrogena SPF50 Sunscreen - Pore-Friendly Protection",
    "Dreft Baby Powder Detergent - Gentle on Baby Clothes",
    "Tide Liquid Detergent - Deep Clean"
]
reviews = [
    "This sunscreen is great for sensitive skin and daily use. It doesn't clog pores and offers great protection!",
    "I love using Dreft for my baby's clothes. It's so gentle and keeps everything smelling fresh."
]

# Compute embeddings using USE
product_embeddings = model(product_descriptions).numpy()  # Convert TensorFlow tensors to NumPy arrays
review_embeddings = model(reviews).numpy()

# Compute cosine similarity between product and review embeddings
cosine_sim = cosine_similarity(review_embeddings, product_embeddings)

# Output similarity scores
for i, review in enumerate(reviews):
    print(f"Review: {review}")
    for j, product in enumerate(product_descriptions):
        print(f" - {product}: Cosine Similarity = {cosine_sim[i][j]:.4f}")


Review: This sunscreen is great for sensitive skin and daily use. It doesn't clog pores and offers great protection!
 - Neutrogena SPF50 Sunscreen - Pore-Friendly Protection: Cosine Similarity = 0.6012
 - Dreft Baby Powder Detergent - Gentle on Baby Clothes: Cosine Similarity = 0.2659
 - Tide Liquid Detergent - Deep Clean: Cosine Similarity = 0.0669
Review: I love using Dreft for my baby's clothes. It's so gentle and keeps everything smelling fresh.
 - Neutrogena SPF50 Sunscreen - Pore-Friendly Protection: Cosine Similarity = 0.1680
 - Dreft Baby Powder Detergent - Gentle on Baby Clothes: Cosine Similarity = 0.6516
 - Tide Liquid Detergent - Deep Clean: Cosine Similarity = 0.2718
